<a href="https://colab.research.google.com/github/vatsasree/CV-Assignment-2/blob/main/CV_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets
from torch.utils.data import DataLoader
import cv2
import numpy as np


In [2]:
train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True,
    transform=torchvision.transforms.ToTensor()
    )

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True,
    transform=torchvision.transforms.ToTensor()
    )


100%|██████████| 9912422/9912422 [00:02<00:00, 3656464.06it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 1192280.45it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:02<00:00, 671654.18it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:01<00:00, 4099.07it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
print(train_set)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [4]:
print(test_set)

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml

# Function to compute cluster centers using K-means
def compute_cluster_centers(descriptors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(descriptors)
    return kmeans.cluster_centers_

# Function to compute histogram of visual words for an image
def compute_histogram(descriptors, cluster_centers):
    histogram = np.zeros(len(cluster_centers))
    for descriptor in descriptors:
        distances = np.linalg.norm(cluster_centers - descriptor, axis=1)
        nearest_cluster = np.argmin(distances)
        histogram[nearest_cluster] += 1
    return histogram

# # Load MNIST dataset
# mnist = fetch_openml('mnist_784', version=1)
# X, y = mnist.data, mnist.target.astype(int)

def create_svm_model(id):
    if id == 1:
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=0.1))
    elif id == 2:
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))
    elif id == 3:
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, gamma='auto'))
    elif id == 4:
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='poly', degree=3, C=1.0))
    elif id == 5:
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=10.0, gamma=0.1))
    elif id == 6:
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='sigmoid', C=0.01, gamma=0.001))
    else:
        # Default model with linear kernel and default C value
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))
    
    return svm_model

def full(num_clusters,id):
    # Initialize SIFT
    X_train, y_train = train_set.data.numpy(), train_set.targets.numpy()

    sift = cv2.SIFT_create()

    # Initialize lists to store descriptors
    descriptors_list = []
    labels_list = []

    for i in range(len(X_train)):
        image = X_train[i]
        keypoints, descriptors = sift.detectAndCompute(image, None)
        # descriptors_list.append(descriptors)
        if descriptors is not None and descriptors.any():
            descriptors_list.append(descriptors)
            labels_list.append(y_train[i])  # Append corresponding label

    # descriptors_list = [descriptors for descriptors in descriptors_list if descriptors is not None and descriptors.any()]

    # Convert descriptors to a single numpy array
    if descriptors_list:
        descriptors_array = np.concatenate(descriptors_list, axis=0)

        # Choose the number of clusters for K-means
        num_clusters = num_clusters

        # Compute cluster centers using K-means
        cluster_centers = compute_cluster_centers(descriptors_array, num_clusters)

        # Compute histograms of visual words for all images
        t_histograms = []
        t_labels = []
        for descriptors, label in zip(descriptors_list, labels_list):
            histogram = compute_histogram(descriptors, cluster_centers)
            t_histograms.append(histogram)
            t_labels.append(label)

        # Convert histograms and labels to numpy arrays
        X_train = np.array(t_histograms)
        y_train = np.array(t_labels)

        # Create a Linear SVM model
        # if id == 1:
        #     svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=0.1))
        # elif if ==2:
        #     svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))
        svm_model = create_svm_model(id)
        # Train the SVM model
        svm_model.fit(X_train, y_train)
    else:
        print("No descriptors found. Unable to create SVM model.")

    # Assuming you have a separate test set, similar to the training set
    X_test, y_test = test_set.data.numpy(), test_set.targets.numpy()

    # Initialize lists to store test descriptors and labels
    test_descriptors_list = []
    test_labels_list = []

    for i in range(len(X_test)):
        image = X_test[i]
        keypoints, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None and descriptors.any():
            test_descriptors_list.append(descriptors)
            test_labels_list.append(y_test[i])

    # Compute histograms of visual words for all test images
    test_histograms = []
    test_labels = []
    for descriptors, label in zip(test_descriptors_list, test_labels_list):
        histogram = compute_histogram(descriptors, cluster_centers)
        test_histograms.append(histogram)
        test_labels.append(label)

    # Convert test histograms and labels to numpy arrays
    X_test = np.array(test_histograms)
    y_test = np.array(test_labels)

    # Make predictions using the trained SVM model
    y_pred = svm_model.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy of the SVM model on the test set:", accuracy)

    import pickle

    filename = 'finalized_model_{}_{}.sav'.format(num_clusters,id)
    pickle.dump(svm_model, open(filename, 'wb'))


    return accuracy



In [7]:
# clusters=[10,20,40,80,160,320]
clusters = [160]
ids=[1,2,3,4,5,6]
accuracies=[]
for num_clusters in clusters:
  for id in ids:
    accuracy = full(num_clusters,id)
    accuracies.append(accuracy)

Accuracy of the SVM model on the test set: 0.7939133986928104
Accuracy of the SVM model on the test set: 0.7953431372549019
Accuracy of the SVM model on the test set: 0.8041258169934641
Accuracy of the SVM model on the test set: 0.7609272875816994
Accuracy of the SVM model on the test set: 0.29074754901960786
Accuracy of the SVM model on the test set: 0.6537990196078431
